# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!pip install python-gmaps

In [2]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather = ("../output_data/cities.csv")
weather_data = pd.read_csv(weather)
weather_data.head(5)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)

In [6]:
gmaps.configure(api_key = g_key)
location = weather_data[['Lat', 'Lng']]
weights = weather_data['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(location, weights=weights))

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
cleaned_data = weather_data

cleaned_data = cleaned_data.loc[(cleaned_data["Max Temp"] <= 85) & (cleaned_data["Max Temp"] > 70)]
cleaned_data = cleaned_data.loc[cleaned_data["Wind Speed"] < 15]
cleaned_data = cleaned_data.loc[cleaned_data["Cloudiness"] == 0]
                                                                                     
cleaned_data = cleaned_data.dropna()

cleaned_data.head(5)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
114,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel = cleaned_data

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel.iterrows():
    lat = row["Lat"]
    lng = row ["Lng"]

    params["location"]= f"{lat}, {lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    results = response ['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel.loc[index, 'Hotel Name'] = results[0]['name']
    
    except (KeyError, IndexError):
        print("Missing result..skipping")
    
    print("-----------")

Closest hotel is Hostal San Isidro Oficial.
-----------
Closest hotel is Escarpas do Lago.
-----------
Closest hotel is Novomar Pousada e Turismo Ltda.
-----------
Closest hotel is Le Monitel.
-----------
Closest hotel is Pousada Porto Praia.
-----------
Closest hotel is La Case Bambou.
-----------
Closest hotel is Dandeli Lake County.
-----------
Closest hotel is Hôtel La Giralda.
-----------
Closest hotel is MOGOK HILL HOTEL.
-----------
Closest hotel is Boboi Beach Lodge.
-----------
Closest hotel is Hotel Relax Inn.
-----------
Closest hotel is تاکسی سرویس آرامش.
-----------
Closest hotel is Marruá Hotel.
-----------
Closest hotel is Norma franco.
-----------
Closest hotel is صالون مها حيدر - Salon Maha.
-----------


In [9]:
hotel.head(5)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41,Hostal San Isidro Oficial
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Escarpas do Lago
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Novomar Pousada e Turismo Ltda
114,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86,Le Monitel
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70,Pousada Porto Praia


In [12]:
# NOTE: Do not change any of the code in this cell
!jupyter nbextension enable --py gmaps
locations = hotel[["Lat", "Lng"]]
m =gmaps.figure()

markers = gmaps.marker_layer(locations)
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel = [info_box_template.format(**row) for index, row in hotel.iterrows()]
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(150, 0, 0, 0.9)',
    stroke_color='rgba(0, 0, 150, 0.9)', scale=3,
    info_box_content=hotel
)
m.add_layer(markers)
m.add_layer(hotel_layer)

m

Traceback (most recent call last):
  File "/Users/trevordoucet/opt/anaconda3/bin/jupyter-nbextension", line 11, in <module>
    sys.exit(main())
  File "/Users/trevordoucet/opt/anaconda3/lib/python3.7/site-packages/jupyter_core/application.py", line 268, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/Users/trevordoucet/opt/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/Users/trevordoucet/opt/anaconda3/lib/python3.7/site-packages/notebook/nbextensions.py", line 983, in start
    super(NBExtensionApp, self).start()
  File "/Users/trevordoucet/opt/anaconda3/lib/python3.7/site-packages/jupyter_core/application.py", line 257, in start
    self.subapp.start()
  File "/Users/trevordoucet/opt/anaconda3/lib/python3.7/site-packages/notebook/nbextensions.py", line 891, in start
    self.toggle_nbextension_python(self.extra_args[0])
  File "/Users/trevordoucet/opt/anac

TypeError: list indices must be integers or slices, not list

In [ ]:
# Add marker layer ontop of heat map


# Display figure
